# Classification: Resampling Techniques for Imbalanced Data - Guided

Imbalanced data is a very common issue that can sometimes cause huge problems for classifier performance. By "imbalanced", we mean that one or more classes is significantly under-represented in the dataset. Sometimes this is fully expected given the domain of the problem at hand - for example, if you were analysing a transaction dataset looking for credit card fraud, you would probably expect to see *many* more instances labelled "not fraud" than "fraud".

Unfortunately, there is no golden gun solution to this problem, but various different strategies have been developed for improving model performance in such cases. You can simply try different algorithms as some are better suited to imbalanced data, you can extend algorithms to assign distinct costs to training examples and thereby give different weights to classes, you can try ensemble methods, or you can try manipulating the datasets itself.

At the data level in particular, there are a few **resampling** techniques that try to balance the classes a little better. We will discuss these in this notebook, as well as their implementation in `imbalanced-learn` or `imblearn`. We will then return to the customer churn problem we tackled in the Introduction work (which used an imbalanced dataset) and see some of these ideas in practice.

### Source Data: Creating an Imbalanced Dataset

In [ ]:
# import the usual libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

To synthesise some data to illustrate these techniques, we can use the `sklearn.datasets.make_classification` class - this is a fairly simple construction that produces normally distributed clusters some specified distance apart, which you can read about in the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html) if you are interested. 

Let's create a binary dataset with 10000 data points, 2 features, a 99:1 class imbalance, and some slight noise (set via the `flip_y` argument) so that the dataset is not *perfectly* linearly separable (in which case an imbalance might not matter much anyway):

In [ ]:
# import make_classification
from sklearn.datasets import make_classification

# construct a binary classification dataset with 2 features, 10000 instances, a 99:1 ratio and slight noise


Note that we also set the `random_state` argument above as, without this, the above code will produce quite a different dataset each time it runs.

This function outputs X and y as `np.array`:

In [ ]:
y

Let's quickly check the class imbalance by converting y to a `pd.Series` and using `value_counts()`:

In [ ]:
# check the class imbalance


Let's combine features and class into a pandas dataframe:

In [ ]:
# combine X and y into a pandas dataframe


In [ ]:
# check the head of the data
df.head()

Let's plot our data on a scatterplot, with `hue` given by the class:

In [ ]:
# produce a scatterplot of the 2 features with hue given by the class


### Random Undersampling

The first resampling technique we will demonstrate is called **random undersampling** - this consists of simply reducing the instances in the majority class (which for us is class 0) by randomly removing some instances, thereby balancing the dataset to a certain degree.

> All of the resampling techniques we will use in this notebook have implementations in the `imbalanced-learn` or `imblearn` library (which is somewhat modelled after `sklearn`) - if you do not currently have it installed, you should install it using pip via a command like `pip install imbalanced-learn`.

Let's implement random undersampling using `RandomUnderSampler` and see how it affects our data. The main argument we want to set is the `sampling_strategy` argument which can take on a number of values as described in the [documentation](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.RandomUnderSampler.html#imblearn.under_sampling.RandomUnderSampler) - for us, we will just set it to a float which will represent the ratio of minority to majority classes after the resampling. All resampling techniques have their own issues, and so you are unlikely to use them to completely balance the classes in practice - we will pick ratio values that are somewhere in the middle throughout.

In [ ]:
# import the function
from imblearn.under_sampling import RandomUnderSampler

# create a RandomUnderSampler that will finish with a 10:1 ratio of classes, and fix the random state


# fit the resampler to our dataset and store it into new variables


Let's now reproduce the previous steps of converting this data into a `pandas` dataframe and producing a scatterplot using `seaborn`:

In [ ]:
# convert the data to a dataframe, rename the target as "class", and plot a scatterplot
df = pd.DataFrame(data = X_undersamp, columns = ["feature_1", "feature_2"])
df['class'] = pd.Series(y_undersamp)

sns.scatterplot(x = df['feature_1'], y = df['feature_2'], hue = df['class'])

As is hopefully clear, the majority class (blue) has fewer instances compared to before, so much so in fact that we can actually see some minority class (orange) instances poking through in the middle of the blue cluster, which were hidden before. 

We can check that this has indeed left the minority class alone and simply reduced the majority class by checking the class imbalance again:

In [ ]:
df['class'].value_counts()

The main advantage of undersampling is that, especially for large datasets, it can reduce both processing time and storage demands by reducing the number of training examples. However, the main issue with this is that instances might be discared that contain potentially useful information for building the classifiers - the remaining sample is chosen randomly, and so there is no guarantee that it is a representative sample of the population, which may lead to bad generalisation.

### Random Oversampling

A contrasting approach is known as **random oversampling** - this randomly duplicates certain instances in the minority class, in order to try and balance the classes a bit. We can implement this using `RandomOverSampler`, which takes the same arguments as `RandomUnderSampler` as you can see in the [documentation](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.RandomOverSampler.html#imblearn.over_sampling.RandomOverSampler).

In [ ]:
# import the function
from imblearn.over_sampling import RandomOverSampler

# create a RandomOverSampler that will finish with a 10:1 ratio of classes, and fix the random state


# fit the resampler to our dataset and store it into new variables


In [ ]:
# convert the data to a dataframe, rename the target as "class", and plot a scatterplot
df = pd.DataFrame(data = X_oversamp , columns = ["feature_1", "feature_2"])
df['class'] = pd.Series(y_oversamp)

sns.scatterplot(x = df['feature_1'], y = df['feature_2'], hue = df['class'])

As we can see, the majority class has not been reduced at all, but some minority class instances have been duplicated - so much so that again we can see some poking through in the middle of majority class cluster in the plot above now.

We can check that oversampling has left the majority class alone, and has simply increased the number of minority class instances:

In [ ]:
df['class'].value_counts()

The main advantage of oversampling is that, unlike undersampling, this does not discared of any training instances and therefore does not risk losing valuable information for classifier building - for this reason it often performs better than undersampling. However, the duplication of minority class instances does increase the likelihood of *overfitting* and so runs the risk of poor generalisation.

### Synthetic Minority Oversampling Technique (SMOTE)

A more sophisticated oversampling technique than simple random oversampling was introduced in a 2002 [paper](https://arxiv.org/pdf/1106.1813.pdf) and utilises the idea of constructing new *synthetic* training examples for the minority class - for this reason the technique is called **synthetic minority oversampling technique (SMOTE)**.

SMOTE creates new synthetic minority instances by drawing *straight lines* between a minority instance and a certain number of nearest minority instance neighbours in feature space, and randomly placing new instances along one of these lines. We can implement it using `SMOTE` which again takes the same arguments as the previous two classes we have utilised.

In [ ]:
# import the function
from imblearn.over_sampling import SMOTE

# create a SMOTE resampler that will finish with a 10:1 ratio of classes, and fix the random state


# fit the resampler to our dataset and store it into new variables


In [ ]:
# convert the data to a dataframe, rename the target as "class", and plot a scatterplot
df = pd.DataFrame(data = X_smote, columns = ["feature_1", "feature_2"])
df['class'] = pd.Series(y_smote)

sns.scatterplot(x = df['feature_1'], y = df['feature_2'], hue = df['class'])

In [ ]:
df['class'].value_counts()

As with random oversampling, this does not risk losing valuable information, and moreover it reduces the risk of overfitting since it generates new synthetic examples rather than merely duplicating existing instances. However, the main problem is that while generating synthetic examples SMOTE does not take into consideration neighboring examples from other classes -  this can result in increase in overlapping of classes and can introduce additional noise as we can see from the above plot. SMOTE is also not very effective for high dimensional data.

### Pipelines

In the original [paper](https://arxiv.org/pdf/1106.1813.pdf) introducing SMOTE, it was suggested that SMOTE be preceeded by undersampling to first trim away some majority class instances - it was argued that this combination produced the best classifier performance. This can be efficiently implemented using `imblearn.pipeline` (which is modelled after `sklearn.pipeline` as will be briefly mentioned in the Online Practice).

In [ ]:
# import the function
from imblearn.pipeline import Pipeline

# set up the undersampler and SMOTE oversampler such that we end up with a 10:1 ratio


# construct the resampling pipeline


# apply the resampling pipeline to the data and store into new variables


In [ ]:
# convert the data to a dataframe, rename the target as "class", and plot a scatterplot
df = pd.DataFrame(data = X_pipe , columns = ["feature_1", "feature_2"])
df['class'] = pd.Series(y_pipe)

sns.scatterplot(x = df['feature_1'], y = df['feature_2'], hue = df['class'])

We can again check the class imbalance as usual:

In [ ]:
df['class'].value_counts()

Clearly we have ended up with less training data here, but the above plot is perhaps a more reasonable representation than that produced by SMOTE alone.

## Returning to our Customer Churn Problem

Let's recall our customer churn problem that we tackled in the Introduction work. Let's quickly reproduce the loading of data, cleaning, and preprocessing that performed there:

In [ ]:
# read in the data
data = pd.read_csv("churn.csv")

# drop unhelpful columns
data = data.drop(['CustomerId', 'Surname', 'RowNumber'], axis = 1)

# make class names clearer
data['Exited'] = data['Exited'].replace([0,1], ['No', 'Yes'])

# split into features (X) and class (y)
X = data.drop('Exited', axis = 1)
y = data['Exited']

from sklearn.model_selection import train_test_split

# perform a train/test split using a 70:30 ratio and fixing the random seed to e.g. 123
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

# perform one-hot encoding on X_train to handle categorical features
X_train = pd.get_dummies(X_train)

# preprocess the test set too
X_test = pd.get_dummies(X_test)

We can check that the train/test split has roughly produced the correct proportions of classes in the train and test sets (for severely imbalanced datasets this might not be the case, and you might need to try a **stratified** split instead):

In [ ]:
y.value_counts()

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

So indeed in this case we see that the class proportions are roughly preserved in the train and test sets.

We now want to rebuild our decision tree model and evaluate as we did in the Introduction work. As discussed previously, accuracy is not always an appropriate measure to use for imbalanced data - other metrics like precision, recall, and f1-score are more suited. 

### Evaluating our Imbalanced Classifier

Before using decision trees, let's quickly use a `LogisticRegression` classifier for now. This is a really fundamental type of classification algorithm that you will learn about in the Online Practice. Let's quickly implement it (in the usual `sklearn` way) first since it doesn't require the extra complexity of grid-searching (unlike decision trees), and is known to handle imbalanced data badly so we will hopefully see a real improvement from resampling. We can evaluate the model via the standard metrics:

In [ ]:
# let's import all the usual metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# import LogisticRegression 


# setup the model


# fit the model to the resampled data


In [ ]:
# evaluate all of the metrics on the test set

print("Original accuracy: ", accuracy_score(y_test, model.predict(X_test)))
print("Original precision: ", precision_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Original recall: ", recall_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Original f1: ", f1_score(y_test, model.predict(X_test), pos_label = 'Yes'))

We see that the accuracy is overly optimistic for imbalanced data but the other metrics give away the underlying issue - this classifier is performing terribly on the other metrics demonstrating how badly the model is performing for the "Yes" class.

Now let's try our resampling combination of undersampling and SMOTE. Note that we never resample the **test set** that we are evaluating our model on - this would change the balance of classes, which is unrepresentative of new data that our classifier will be predicting on when it is deployed.

In [ ]:
# set up the undersampler and SMOTE oversampler


# construct the pipeline
steps = [('u', under), ('o', over)]
pipeline = Pipeline(steps = steps)

# reample the data using the pipeline


In [ ]:
# setup the model


# fit the model to the resampled data


In [ ]:
# again evaluate all of the metrics on the test set
print("Resampled accuracy: ", accuracy_score(y_test, model.predict(X_test)))
print("Resampled precision: ", precision_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Resampled recall: ", recall_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Resampled f1: ", f1_score(y_test, model.predict(X_test), pos_label = 'Yes'))

Other than a slight drop in accuracy, this leads to a dramatic improvement in performance on the whole (though clearly still far from perfect performance).

### Using Decision Trees

Let's now repeat this analysis using decision trees, like we originally did in the Introduction work. Decision trees are actually inherently better at handling imbalanced data, but we can check to see if we can still get any model improvement.

Strictly speaking, we should repeat the grid search to choose our hyperparameter values when performing resampling, since the dataset will be altered. This introduces some extra complexity, since it is preferrable to use **stratified** k-folds when cross-validating, so that the class balances are preserved in each fold - we leave this as optional stretch material. For now, we will simply choose the "best" tree we found from the Introduction, which you might recall had `criterion = 'gini'` and `max_depth = 5`, and see if resampling improves the performance at all.

In [ ]:
# import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# setup the tree model with critetion = 'gini' and max_depth = 5


# fit the model to the training data


# compute the metrics on the test set
print("Original accuracy: ", accuracy_score(y_test, model.predict(X_test)))
print("Original precision: ", precision_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Original recall: ", recall_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Original f1: ", f1_score(y_test, model.predict(X_test), pos_label = 'Yes'))

Now let's repeat our resampling pipeline and evaluate again:

In [ ]:
# setup the undersample and SMOTE oversampler


# construct the pipeline
steps = [('u', under), ('o', over)]
pipeline = Pipeline(steps = steps)

# run the pipeline


# setup the model


# fit the model to the resampled data


print("Resampled accuracy: ", accuracy_score(y_test, model.predict(X_test)))
print("Resampled precision: ", precision_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Resampled recall: ", recall_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Resampled f1: ", f1_score(y_test, model.predict(X_test), pos_label = 'Yes'))

When we look it at like this we see that the classifier is performing quite differently: 

- The accuracy is about the same
- The precision has dropped by about 10%
- Notably, the recall has come up by about 13%
- Importantly, the f1-score has also come up by about 6%

There is no such thing as a free lunch, and these resampling techniques don't just solve the problem of imbalanced data for you - as expected from what we saw in the Introduction work, increasing the recall generally leads to a reduction in precision and vice versa, in line with the **precision-recall tradeoff** that we have mentioned previously. Depending on the case and the data at hand, these techniques can often improve classifier performance in the way that might matter to you for the problem you are solving, and so they are worth trying.

### Extra: Grid-searching with Decision Trees

Let's repeat the `GridSearchCV` we did in the Introduction work to find the hyperparameters, but with one slight difference. Rather than simply creating k-folds randomly, we will create **stratified** folds so that the class proportions are maintained in each fold. This can be implemented using the `RepeatedStratifiedKFold` class from `sklearn` which can then be passed as the `cv` argument in `GridSearchCV`, rather than passing a simple integer as we did before (to specify the number of folds). We will use `roc_auc` as the scoring (the area under the ROC curve) rather than accuracy in the `GridSearchCV` - we will cover these concepts fully in the Online Practice.

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [ ]:
# define the parameter grid
param_grid = [
        {'criterion': ['gini', 'entropy'], 
         'max_depth': [2, 5, 7, 9, 11]},
]

# setup the model with all arguments as default
model = DecisionTreeClassifier()

cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

# pass the model, cv, and the param_grid to the grid search
grid_search = GridSearchCV(model, param_grid, cv = cv, scoring = 'roc_auc')

# fit the grid search to the training set
grid_search.fit(X_train, y_train)

# return the hyperparameter values of the best model
grid_search.best_params_

Mean score (AUC) of best estimator from cross-validation:

In [ ]:
grid_search.best_score_

In [ ]:
model = grid_search.best_estimator_

Let's see how it performs on the test set:

In [ ]:
model.fit(X_train, y_train)

We can summarise the classifier performance more fully by looking at the precision, recall and f1-score:

In [ ]:
print("Original accuracy: ", accuracy_score(y_test, model.predict(X_test)))
print("Original precision: ", precision_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Original recall: ", recall_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Original f1: ", f1_score(y_test, model.predict(X_test), pos_label = 'Yes'))

#### Using Resampling

Now let's try our combination of undersampling and SMOTE. Note that we never resample the **test set** that we are evaluating our model on - this would change the balance of classes, which is unrepresentative of new data that our classifier will be predicting on when it is deployed.

In [ ]:
under = RandomUnderSampler(sampling_strategy = 0.3, random_state = 1)
over = SMOTE(sampling_strategy = 0.6, random_state = 1)

steps = [('u', under), ('o', over)]
pipeline = Pipeline(steps = steps)

X_pipe, y_pipe = pipeline.fit_resample(X_train, y_train)

Let's now repeat the grid search with our resampled training set:

In [ ]:
# define the parameter grid
param_grid = [
        {'criterion': ['gini', 'entropy'], 
         'max_depth': [2, 5, 7, 9, 11]},
]

# setup the model with all arguments as default
model = DecisionTreeClassifier()

cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

# pass the model and the param_grid to the grid search
grid_search = GridSearchCV(model, param_grid, cv = cv, scoring = 'roc_auc')

# fit the grid search to the training set
grid_search.fit(X_pipe, y_pipe)

# return the hyperparameter values of the best model
grid_search.best_params_

Mean score (AUC) of best estimator from cross-validation:

In [ ]:
grid_search.best_score_

and so see that this score has gone up a bit.

In [ ]:
model = grid_search.best_estimator_

Let's see how it performs on the **test set** however (which recall is totally "new" data that was not resampled at all, and so retains the original class imbalance):

In [ ]:
model.fit(X_pipe, y_pipe)

We can see exactly how our classifier is performing differently now by looking at the various metrics:

In [ ]:
print("Resampled accuracy: ", accuracy_score(y_test, model.predict(X_test)))
print("Resampled precision: ", precision_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Resampled recall: ", recall_score(y_test, model.predict(X_test), pos_label = 'Yes'))
print("Resampled f1: ", f1_score(y_test, model.predict(X_test), pos_label = 'Yes'))

We see a similar story to what we found with the best tree from the Introduction work that we used before - a slight drop in precision, but an increase in recall and f1-score.

**Ensemble methods**, which you will learn about in your next Classification module, are another way to improve performance for imbalanced data, and sometimes a combination of resampling techniques and ensemble methods are the best strategy.